Preliminaries for .toml files

In [6]:
# ] add package#branch

In [12]:
cd("../TestDrive")

In [13]:
] activate . 

In [14]:
] up

  Updating registry at `C:\Users\valer\.julia\registries\General`
  Updating git-repo `https://github.com/JuliaRegistries/General.git`
Fetching: [========================================>]  100.0 %.0 %  Updating git-repo `https://github.com/JuliaOpt/GLPK.jl.git`
  Updating git-repo `https://github.com/JuliaOpt/JuMP.jl.git`
Fetching: [========================================>]  100.0 %.0 %0.0 %Fetching: [======================================>  ]  93.3 %  Updating git-repo `https://github.com/JuliaOpt/Ipopt.jl.git`
  Updating git-repo `https://github.com/JuliaOpt/MathOptInterface.jl.git`
 Resolving package versions...
  Updating `C:\Users\valer\Dropbox\Berkeley\Code\Modelling_TrialRuns\TestDrive\Project.toml`
  [4076af6c] ↑ JuMP v0.19.2 #master (https://github.com/JuliaOpt/JuMP.jl.git) ⇒ v0.20.0 #master (https://github.com/JuliaOpt/JuMP.jl.git)
  Updating `C:\Users\valer\Dropbox\Berkeley\Code\Modelling_TrialRuns\TestDrive\Manifest.toml`
  [4076af6c] ↑ JuMP v0.19.2 #master (https://githu

In [15]:
using Plots
gr()
;

┌ Info: Precompiling Plots [91a5bcdd-55d7-5caf-9e0b-520d859cae80]
└ @ Base loading.jl:1186
ERROR: LoadError: UndefVarError: ffmpeg not defined
Stacktrace:
 [1] top-level scope at none:0
 [2] include at .\boot.jl:326 [inlined]
 [3] include_relative(::Module, ::String) at .\loading.jl:1038
 [4] include(::Module, ::String) at .\sysimg.jl:29
 [5] top-level scope at none:2
 [6] eval at .\boot.jl:328 [inlined]
 [7] eval(::Expr) at .\client.jl:404
 [8] top-level scope at .\none:3
in expression starting at C:\Users\valer\.julia\packages\FFMPEG\9JQpZ\src\FFMPEG.jl:55
ERROR: LoadError: Failed to precompile FFMPEG [c87230d0-a227-11e9-1b43-d7ebe4e7570a] to C:\Users\valer\.julia\compiled\v1.1\FFMPEG\TGvga.ji.
Stacktrace:
 [1] error(::String) at .\error.jl:33
 [2] compilecache(::Base.PkgId, ::String) at .\loading.jl:1197
 [3] _require(::Base.PkgId) at .\loading.jl:960
 [4] require(::Base.PkgId) at .\loading.jl:858
 [5] require(::Module, ::Symbol) at .\loading.jl:853
 [6] include at .\boot.jl:326 [in

ErrorException: Failed to precompile Plots [91a5bcdd-55d7-5caf-9e0b-520d859cae80] to C:\Users\valer\.julia\compiled\v1.1\Plots\ld3vC.ji.

Main file 

In [16]:
include("src/TestDrive.jl")

population_model_network (generic function with 1 method)

Specify migration (could also select from pre-existing migration matrices in data files)

In [17]:
# User-specified fake migration matrices 

migrate1 = rand((6*(4+8+6+1+6)), 2, 2) 
migrate2 = rand((6*(4+8+6+1+6)), 2, 2)
;

Define insect details, genetic construct, and population structure for experimentation

In [18]:
# This uses pre-existing stage data (aedes) and genetics (MCR) -> if desired, user could specify new. 

# User-named nodes  
firstnode = :FirstNode
secondnode = :SecondNode

# User-constructed nodes: [name, gene_data, stages, location, migration_matrix]
example_nodes = [Node(:FirstNode, genetics_mcr, stages_aedes, temp_none, (37.87, 122.27), migrate1),
    Node(:SecondNode, genetics_mcr, stages_aedes, temp_none, (35.87, 120.27), migrate2)]

# User-constructed network: [nodes, total states (# genotypes x states in each lifestage)] 
example_network = Network(example_nodes, 150)
;

"True" equilibrium: use analytic guesses in NLSolve

In [19]:
# NB can also initialize each stage separately 
# (for now - working on way to do all at once) if want to have different genotypes in different stages
# init_node!(desired_node, gene_index)
u0 = init_node!(example_nodes[1], 4)
;

DiffEq: ODE without controls 

In [20]:
tspan = (0.0,300.0) 
controls = [0.0] # necessary even when not using controls bc of the way function is written 
ode_model = diffeq.ODEProblem(population_model_node, u0, tspan, (example_nodes[1], controls))

ODEProblem with uType Array{Float64,2} and tType Float64. In-place: true
timespan: (0.0, 300.0)
u0: [1.76687e-18 -1.73706e-18 … 0.0 0.0; 1.68273e-18 -1.65435e-18 … 0.0 0.0; … ; 0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0]

In [21]:
alg = OrdinaryDiffEq.Tsit5() 
sol_ode = diffeq.solve(ode_model, alg)
;

Analyze result and plot

In [22]:
sol_ode.u[end][:,4]


25-element Array{Float64,1}:
 7625.200711632755        
 7262.0959284002465       
 6916.281847206621        
 6586.9351010350065       
 1498.2853517561823       
  908.8192733559194       
  551.2556065697543       
  334.3831589441913       
  202.81786221895192      
  123.03099310503568      
   74.62254090787292      
   45.261394984956006     
  114.96076760597006      
  109.48320192694933      
  104.27054349867802      
   99.30510191607539      
   94.57632751892713      
   90.07268205756431      
  500.4037962568068       
   -1.8653440482592828e-30
   -2.5678786029510796e-14
    0.0                   
  500.40379625680686      
    0.0                   
    0.0                   

In [23]:
plot(sol_ode, title = "MCR Population at Equilibrium") 

UndefVarError: UndefVarError: plot not defined

DiffEq: ODE with controls 

In [24]:
tspan = (0.0,300.0) 
controls = [0.0]

# Construct ODE control problem 
control_model = diffeq.ODEProblem(population_model_node, u0, tspan, (example_nodes[1], controls))

ODEProblem with uType Array{Float64,2} and tType Float64. In-place: true
timespan: (0.0, 300.0)
u0: [1.76687e-18 -1.73706e-18 … 0.0 0.0; 1.68273e-18 -1.65435e-18 … 0.0 0.0; … ; 0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0]

Choose algorithm and solve

In [25]:
#alg = Vern8()
#alg = OrdinaryDiffEq.Tsit5() 
alg = Sundials.CVODE_Adams()
#alg = CVODE_BDF()

sol_control = diffeq.solve(control_model, alg, callback = cbs, tstops = tstop)
;

Spot-check results 

In [26]:
sol_control.u[end]


25×6 Array{Float64,2}:
 7631.7        1.11067     0.0  0.00618558   0.0  0.0
 7268.69       1.13906     0.0  0.00644327   0.0  0.0
 6922.98       1.16805     0.0  0.00671169   0.0  0.0
 6593.73       1.19766     0.0  0.00699129   0.0  0.0
 1499.5        0.277066    0.0  0.00162311   0.0  0.0
  909.332      0.170918    0.0  0.00100483   0.0  0.0
  551.452      0.10544     0.0  0.000622093  0.0  0.0
  334.415      0.0650439   0.0  0.000385126  0.0  0.0
  202.782      0.0401212   0.0  0.000238407  0.0  0.0
  123.006      0.0247573   0.0  0.00014764   0.0  0.0
   74.5342     0.0152584   0.0  9.13172e-5   0.0  0.0
   45.2836     0.00943233  0.0  5.66555e-5   0.0  0.0
  114.819      0.0257336   0.0  0.000157068  0.0  0.0
  109.378      0.0263792   0.0  0.000163637  0.0  0.0
  104.156      0.0270275   0.0  0.000170413  0.0  0.0
   99.19       0.0276902   0.0  0.00017748   0.0  0.0
   94.459      0.0283652   0.0  0.000184835  0.0  0.0
   89.9534     0.029053    0.0  0.000192492  0.0  0.0
  498

Plot

In [27]:
# Question - full plotting routines here in Example file or elsewhere? Check PVCR example. 
# Question - use other plotting packages to make it nice; which? and choose appropriate backend. 

plot(sol_control, title = "MCR Population with Releases")

UndefVarError: UndefVarError: plot not defined

In [28]:
males_HH = [val[19,1] for val in sol_control.u]
males_Hh = [val[19,2] for val in sol_control.u]
males_HR = [val[19,3] for val in sol_control.u]
males_hh = [val[19,4] for val in sol_control.u]
males_hR = [val[19,5] for val in sol_control.u]
males_RR = [val[19,6] for val in sol_control.u]
;

In [29]:
plot(sol_control.t, males_HH, label = "Males_HH", title = "MCR Males by Genotype")
plot!(sol_control.t, males_Hh, label = "Males_Hh")
plot!(sol_control.t, males_HR, label = "Males_HR")
plot!(sol_control.t, males_hh, label = "Males_hh")
plot!(sol_control.t, males_hR, label = "Males_hR")
plot!(sol_control.t, males_RR, label = "Males_RR")

UndefVarError: UndefVarError: plot not defined

In [30]:
# list comprehensions return arrays; hence brackets
sumfemales_HH = [sum(val[20,:]) for val in sol_control.u]   
sumfemales_Hh = [sum(val[21,:]) for val in sol_control.u]
sumfemales_HR = [sum(val[22,:]) for val in sol_control.u]
sumfemales_hh = [sum(val[23,:]) for val in sol_control.u]
sumfemales_hR = [sum(val[24,:]) for val in sol_control.u]
sumfemales_RR = [sum(val[25,:]) for val in sol_control.u]
;

In [31]:
plot(sol_control.t, sumfemales_HH, label = "HH", title = "MCR Females by Genotype")
plot!(sol_control.t, sumfemales_Hh, label = "Hh")
plot!(sol_control.t, sumfemales_HR, label = "HR")
plot!(sol_control.t, sumfemales_hh, label = "hh")
plot!(sol_control.t, sumfemales_hR, label = "hR")
plot!(sol_control.t, sumfemales_RR, label = "RR")

UndefVarError: UndefVarError: plot not defined